In [2]:
import torch
import torch.nn as nn

In [3]:
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your(x^1)
[0.55, 0.87, 0.66], # journey(x^2)
[0.57, 0.85, 0.64], # starts(x^3)
[0.22, 0.58, 0.33], # with(x^4)
[0.77, 0.25, 0.10], # one(x^5)
[0.05, 0.80, 0.55]] # step(x^6)
)

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

In [14]:
class CausalAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, context_length: int, dropout: int, qkv_bias=False):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.context_length = context_length
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key= nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )


    def forward(self, x):
        # here x is our input torch tensor batch(embedding of a sentence tokens)
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores = attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values   # here i get context_vectors
        return context_vec

In [15]:
causal_attention = CausalAttention(d_in=3, d_out=2, context_length=6, dropout=0.1)

In [16]:
context_vec = causal_attention(batch)

In [17]:
print(context_vec.shape)
print(context_vec)

torch.Size([2, 6, 2])
tensor([[[ 0.4761, -0.6235],
         [ 0.3204, -0.3609],
         [ 0.2722, -0.2837],
         [ 0.2158, -0.2032],
         [ 0.1795, -0.2242],
         [ 0.1677, -0.1560]],

        [[ 0.4761, -0.6235],
         [ 0.3204, -0.3609],
         [ 0.2722, -0.2837],
         [ 0.2158, -0.2032],
         [ 0.1795, -0.2242],
         [ 0.0980, -0.0647]]], grad_fn=<UnsafeViewBackward0>)
